In [1]:
from utt_encoding import UtteranceEncoding
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
import pandas as pd
from IPython.display import clear_output

In [2]:
def string_prop(dp_content):
    dp_content = dp_content.replace("!=", "does not equal").replace("=", "equals").\
    replace("<", "is less than").replace(">", "is more than").\
    replace("+", "plus").replace("10", "ten").replace("20", "twenty").replace("30", "thirty").\
    replace("40", "forty").replace("50","fifty").replace("red", "red block").replace("blue", "blue block").\
    replace("yellow", "yellow block").replace("purple", "purple block").replace("green", "green block")
    return dp_content

In [3]:
def unstring_prop(dp_content):
    dp_content = dp_content.replace("equals","=").replace("does not equal","!=").\
    replace("is less than","<").replace("is more than",">").\
    replace("plus","+").replace("ten","10").replace("twenty","20").replace("thirty","30").\
    replace("forty","40").replace("fifty","50").replace(" block","")
    return dp_content

In [4]:
props = list(pd.read_csv("Data\goldenFiles\NormalizedList.csv")["Propositions"])
for id, prop in enumerate(props):
    props[id] = string_prop(prop)

In [5]:
utterance_encoder = UtteranceEncoding(props)
clear_output(wait=False)

In [6]:
colors = ['red', 'blue', 'green', 'purple', 'yellow']

In [7]:
def calculate_iou(common_ground, actual_common_ground):
    # Split by ',' and strip whitespace from each element
    common_ground_elements = set([element.strip() for element in common_ground.split(',')])
    actual_common_ground_elements = set([element.strip() for element in actual_common_ground.split(',')])
    # Calculate intersection and union
    intersection = common_ground_elements & actual_common_ground_elements
    union = common_ground_elements | actual_common_ground_elements
    # Calculate IoU
    iou = len(intersection) / len(union) if union else 0
    return iou

In [8]:
l = []
top1 = []
top3 = []
top5 = []
iou = []
acc_1, acc_3, acc_5, acc_all = 0, 0, 0, 0
jacc_1, jacc_3, jacc_5, jacc_all =[], [], [], []
data_file = pd.read_csv("Data\goldenFiles\preprocessedTrainingData.csv")
data_file = data_file[data_file["Label"]==1]

for row_id, utterance in enumerate(data_file["Transcript"]):
    selected_colors = []
    for color in colors:
        if color in utterance.lower():
            selected_colors.append(color)
    tokenized_text, seq_embedding, pooled_embedding, hidden_states = \
                            utterance_encoder.get_sentence_embedding(utterance,
                            utterance_encoder.bert_base_uncased_tokenizer,
                            utterance_encoder.bert_base_uncased_model)
    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0), dim=1).permute(1,0,2)
    sum_embeddings = utterance_encoder.get_token_embeddings(token_embeddings, 4)
    utterance_embedding = torch.mean(torch.stack(sum_embeddings[:-1]),axis=0)
    similarities = []
    for prop in utterance_encoder.prop_embs:
        # for color in selected_colors:
        #     if color in prop:
                similarities.append(cos_sim(utterance_embedding.reshape(1,-1),\
                    utterance_encoder.prop_embs[prop].reshape(1,-1))[0][0])
    # print(np.max(np.array(similarities)),utterance_encoder.props[np.argmax(np.array(similarities))])
    true_prop = data_file.iloc[row_id]["Common Ground"]
    # Top 1 acc
    dp_content = utterance_encoder.props[np.argmax(np.array(similarities))]
    dp_content = unstring_prop(dp_content)
    if dp_content == true_prop:
        acc_1 += 1
        top1.append(row_id)
    iou.append(calculate_iou(dp_content, true_prop))

    # Top 3 acc
    for prop_3 in np.array(similarities).argsort()[::-1][:3]:
        dp_content = utterance_encoder.props[prop_3]
        dp_content = unstring_prop(dp_content)
        if dp_content == true_prop:
            acc_3 += 1
            top3.append(row_id)
            
    # Top 5 acc
    old = acc_5
    for prop_5 in np.array(similarities).argsort()[::-1]:
        dp_content = utterance_encoder.props[prop_5]
        dp_content = unstring_prop(dp_content)
        if dp_content == true_prop:
            acc_5 += 1
            top5.append(row_id)
    if acc_5 == old:
        l.append(row_id)

In [9]:
len(l)

3

In [10]:
l

[79, 80, 85]

In [11]:
print(top1)
print(top3)
print(top5)

[1, 3, 6, 12, 30, 49, 50, 55, 57, 64, 67, 91, 110]
[0, 1, 3, 6, 12, 15, 21, 30, 32, 37, 49, 50, 55, 57, 64, 65, 67, 86, 91, 92, 94, 110]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126]


In [12]:
print(acc_1)
print(acc_3)
print(acc_5)

13
22
124


In [14]:
iou

[0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.3333333333333333,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.25,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.25,
 0.0,
 0.25,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.16666666666666666]

In [13]:
sum(iou)/len(iou)

0.12427821522309711